In [ ]:
# This file is a part of https://github.com/pawansharmaaaa/Lip_Wise/ repository.
import gradio as gr

# Custom Modules
import infer
import file_check

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

# Create interface
inputs_for_image = [
    gr.Image(type="filepath", label="Image"),
    gr.Audio(type="filepath", label="Audio"),
    gr.Number(value=0, label="Padding: Increase if getting black outlines"),
    gr.Checkbox(label = "Perform 3D_alignment?"),
    gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer"),
    gr.Slider(minimum=1, maximum=60, step=1, value=25, label="FPS"),
    gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False),
    gr.Slider(minimum=0.0, maximum=1.0, step=0.1, value=0.3, label="Weight"),
    gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False),
    gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus")
]
output_for_image = gr.Video(sources='upload', label="Output")

image_Interface = gr.Interface(fn=infer.infer_image, inputs=inputs_for_image, outputs=output_for_image)

inputs_for_video = [
    gr.Video(sources='upload',label="Video"),
    gr.Audio(type="filepath", label="Audio"),
    gr.Number(value=0, label="Padding: Increase if getting black outlines"),
    gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer"),
    gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False),
    gr.Slider(minimum=0.0, maximum=1.0, step=0.1, value=0.3, label="Weight"),
    gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False),
    gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus")
]
output_for_video = gr.Video(sources='upload', label="Output")

video_Interface = gr.Interface(fn=infer.infer_video, inputs=inputs_for_video, outputs=output_for_video)

# Run interface
ui = gr.TabbedInterface([image_Interface, video_Interface], ['Process Image', 'Process Video'],title="Lip-Wise",theme='gradio/monochrome')

ui.launch()

# New UI

In [ ]:
WEIGHTS = '''
For CodeFormer:
    \n\t0 for better quality, 1 for better identity
\nFor GFPGAN:
    \n\t1 for better quality, 0 for better identity
'''

In [ ]:
# This file is a part of https://github.com/pawansharmaaaa/Lip_Wise/ repository.
import gradio as gr

# Custom Modules
import infer
import file_check

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

def render_dd(bg_upscale):
    
    return gr.Dropdown(
                        choices=bg_upscalers,
                        label="REALESRGAN Model",
                        value="RealESRGAN_x2plus",
                        info="Choose the model to use for upscaling the background.",
                        # Initially disabled and hidden
                        visible=bg_upscale
                    )

def render_weight(face_restorer):
    if face_restorer == "CodeFormer":
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="CodeFormer Weight",
                            info="0 for better quality, 1 for better identity."
                        )
    else:
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.5,
                            label="GFPGAN Weight",
                            info="0 for better identity, 1 for better quality."
                        )

# Create interface

with gr.Blocks() as ui:
    gr.Markdown('''
        <div style="display: flex; justify-content: center;">
            <img src = "https://github.com/pawansharmaaaa/Lip_Wise/assets/56242483/cca4987a-18e5-49f1-9eba-710f3c0655a7">
        </div>
        ''')
    with gr.Tab(label="Process Image"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("# INPUTS")
                with gr.Group():
                    image_input = gr.Image(type="filepath", label="Image", container=True)
                    audio_input = gr.Audio(type="filepath", label="Audio", container=True, scale=0.5)

                with gr.Accordion(label="OPTIONS", open=True):
                    with gr.Group():
                        with gr.Row():
                            fps = gr.Slider(minimum=1, maximum=60, step=1, value=25, label="FPS", info="Desired Frames per second (FPS) of the output video.")
                            padding = gr.Slider(minimum=0, maximum=60, step=1, value=0, label="Padding", info="Increase if getting black outlines. The Value is in Pixels.")
                        with gr.Row():
                            alignment = gr.Checkbox(label = "Perform 3D_alignment", info = "This will improve the quality of the lip sync, but the output will be different from the original video.")
                            upscale_bg = gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False, info="This will improve the quality of the video, but will take longer to process.")
                        with gr.Row():
                            face_restorer = gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer", info="GFPGAN is faster, but CodeFormer is more accurate.", interactive=True)
                            bg_model = gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus", info="Choose the model to use for upscaling the background.", visible=False)
                        with gr.Row():
                            with gr.Column():
                                mel_step_size = gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False)
                                weight = render_weight(face_restorer)
                                upscale_bg.select(render_dd, upscale_bg, bg_model)
                                face_restorer.select(render_weight, face_restorer, weight)
                
                process = gr.Button(value="Process Image", variant="primary")


            with gr.Column():
                gr.Markdown("# OUTPUT")
                image_output = gr.Video(sources='upload', label="Output")

                process.click(infer.infer_image, [image_input, audio_input, padding, alignment, face_restorer, fps, mel_step_size, weight, upscale_bg, bg_model], [image_output])

    with gr.Tab(label="Process Video"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("# INPUTS")
                
                with gr.Group():
                    video_input = gr.Video(sources='upload',label="Video")
                    audio_input = gr.Audio(type="filepath", label="Audio")
                
                with gr.Accordion(label="OPTIONS", open=True):
                    with gr.Group():
                        with gr.Column():
                            padding = gr.Slider(minimum=0, maximum=60, step=1, value=0, label="Padding", info="Increase if getting black outlines. The Value is in Pixels.")
                            face_restorer = gr.Radio(["GFPGAN", "CodeFormer"], value='CodeFormer', label="Face Restorer", info="GFPGAN is faster, but CodeFormer is more accurate.")
                            mel_step_size = gr.Number(value=16, label="Mel Step Size", interactive=False, visible=False)
                            weight = render_weight(face_restorer)
                        with gr.Column():
                            upscale_bg = gr.Checkbox(label = "Upscale Background with REALESRGAN", value=False, info="This will improve the quality of the video, but will take longer to process.")
                            bg_model = gr.Dropdown(choices=bg_upscalers, label="REALESRGAN Model", value="RealESRGAN_x2plus", info="Choose the model to use for upscaling the background.", visible=False)
                            
                            upscale_bg.select(render_dd, upscale_bg, bg_model)
                            face_restorer.select(render_weight, face_restorer, weight)
                    
                process = gr.Button(value="Process Video", variant="primary")

            with gr.Column():
                gr.Markdown("# OUTPUT")
                video_output = gr.Video(sources='upload', label="Output")

                process.click(infer.infer_video, [video_input, audio_input, padding, face_restorer, mel_step_size, weight, upscale_bg, bg_model], [video_output])

    with gr.Tab(label="Guide"):
        with gr.Accordion(label="Tips For Better Results", open=True):
            gr.Markdown(
            """
            > - Optimal performance is achieved with a **clear image** featuring a person facing the camera, regardless of head angle. However, avoid **tilting in the z-direction** (3D-alignment can address this with certain considerations).
            > - Ensure the image contains only **one person** with a prominently visible face.
            > - Clear audio devoid of background noise enhances results significantly.
            > - Note that **higher image resolution** necessitates **additional processing time**.
            """,
            line_breaks=True)

        with gr.Accordion(label="Model Selection", open=True):
            gr.Markdown(
            """
            > ##### **CODEFORMER:**
            >**Recommended Weight:** `0 for better quality, 1 for better identity.`
            >>- CodeFormer employs a transformative architecture to restore facial features.
            >>- While relatively slower, it boasts **higher accuracy**.
            >>- Generally delivers superior results while **preserving skin texture**.
            >>- In cases of peculiar artifacts, especially around the nose, consider using GFPGAN.
            
            > ##### **GFPGAN:**
            >**Recommended Weight:** `0 for better identity, 1 for better quality.`
            >>- GFPGAN, a faster model, relies on a GAN-based framework for facial restoration.
            >>- Suggested for use **when CodeFormer exhibits undesirable artifacts**.
            >>- However, it often sacrifices skin texture fidelity.
            """,
            line_breaks=True)

        with gr.Accordion(label="3D Alignment", open=True):
            gr.Markdown(
            """
            > Enabling this feature **transforms** the image to ensure the person faces the camera directly. While enhancing lip sync quality, the output may diverge from the original video.
            """,
            line_breaks=True)

        with gr.Accordion(label="Background Upscaling", open=True):
            gr.Markdown(
            """
            > - Activating this feature **enhances video quality** but prolongs processing time.
            > - For most scenarios, RealESRGAN_x2plus is preferable due to its comparative speed.
            > - Optimal results are achieved when combined with CodeFormer, except in cases of nose-related artifacts.
            > - This feature effectively **eliminates video flickering**.
            """,
            line_breaks=True)

ui.launch()

# Building theme

In [ ]:
gr.themes.builder()

In [ ]:
import gradio as gr
import file_check

def render_dd(bg_upscale):
    
    return gr.Dropdown(
                        choices=bg_upscalers,
                        label="REALESRGAN Model",
                        value="RealESRGAN_x2plus",
                        info="Choose the model to use for upscaling the background.",
                        # Initially disabled and hidden
                        visible=bg_upscale
                    )

def render_weight(face_restorer):
    if face_restorer == "CodeFormer":
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="CodeFormer Weight"
                        )
    else:
        return gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.5,
                            label="GFPGAN Weight"
                        )

bg_upscalers = list(file_check.REAL_ESRGAN_MODEL_URL.keys())

# Assuming you have necessary imports and definitions for bg_upscalers
with gr.Blocks() as ui:
    with gr.Tab(label="Process Video"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("# INPUTS")
                with gr.Group():
                    video_input = gr.Video(sources='upload', label="Video")
                    audio_input = gr.Audio(type="filepath", label="Audio")
                with gr.Group():
                    with gr.Row():
                        padding = gr.Slider(
                            minimum=1,
                            maximum=60,
                            step=1,
                            value=0,
                            label="Padding",
                            info="Increase if getting black outlines. The Value is in Pixels."
                        )
                        face_restorer = gr.Radio(
                            ["GFPGAN", "CodeFormer"],
                            value='CodeFormer',
                            label="Face Restorer",
                            info="GFPGAN is faster, but CodeFormer is more accurate."
                        )
                        mel_step_size = gr.Number(
                            value=16,
                            label="Mel Step Size",
                            interactive=False,
                            visible=False
                        )
                        weight = gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            value=0.3,
                            label="Restorer Weight"
                        )
                    with gr.Row():
                        upscale_bg = gr.Checkbox(
                            label="Upscale Background with REALESRGAN",
                            value=False,
                            info="This will improve the quality of the video, but will take longer to process."
                        )
                        bg_model = gr.Dropdown(
                            choices=bg_upscalers,
                            label="REALESRGAN Model",
                            value="RealESRGAN_x2plus",
                            info="Choose the model to use for upscaling the background.",
                            # Initially disabled and hidden
                            visible=False
                        )
                        upscale_bg.select(render_dd, upscale_bg, bg_model)

            with gr.Column():
                gr.Markdown("# OUTPUT")
                image_output = gr.Video(sources='upload', label="Output")

    live = True

ui.launch()

# ... rest of your Gradio interface



In [6]:
import gradio as gr

# Custom CSS
custom_css = """
html, body {
  font-family: JetBrains Mono, monospace;
  margin: 0;
  padding: 0;
}

.gradio-header {
  background-color: #ffffff;
  padding: 10px 20px;
  display: flex;
  justify-content: space-between;
  align-items: center;
}

.gradio-header h1 {
  font-size: 20px;
  margin: 0;
}

.gradio-input {
  margin-bottom: 15px;
}
"""

with gr.Blocks(css=custom_css, head=gr.HTML("""
        <h1>Calculator App</h1>
    """)) as demo:
    # Custom header

    # Inputs
    num1 = gr.Number(label="Number 1")
    num2 = gr.Number(label="Number 2")

    # Calculation
    def add(n1, n2):
        return n1 + n2

    # Output
    output = gr.Number(label="Result")

    num1.change(add, [num1, num2], output)
    num2.change(add, [num1, num2], output)

demo.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
